# Importing the Libraries here

In [ ]:
!python -m spacy download en_core_web_trf
!python -m spacy download en_core_web_sm
!python -m spacy download en_core_web_md

!pip install --upgrade transformers
!pip install --upgrade sentence-transformers
!pip install bertopic # Install bertopic library
!pip install selenium


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 457.4/457.4 MB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.3/236.3 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.6/731.6 kB 22.5 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_trf')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 77.0 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.
   

In [ ]:
# Importing required libraries
import unicodedata
import time
import re
import string
import csv
import requests
import io
from itertools import combinations
from collections import defaultdict, Counter

# Data manipulation and visualization
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Uncomment if seaborn is needed for visualization
# import seaborn as sns

# Text processing and natural language processing (NLP)
import re
import string
import unicodedata
from collections import defaultdict, Counter
from itertools import combinations

# NLTK for NLP
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.corpus import wordnet

# Download necessary NLTK datasets
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('vader_lexicon')

# spaCy for advanced NLP (using BERT-based transformer model)
import spacy
#nlp = spacy.load("en_core_web_trf")

# Gensim for topic modeling (LDA)
import gensim
from gensim import corpora
from gensim.models import LdaModel

# BERTopic for topic modeling using BERT embeddings
from bertopic import BERTopic
from bertopic.vectorizers import ClassTfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer

# Scikit-learn for machine learning models and feature extraction
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Selenium WebDriver for web scraping
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# Selenium Chrome options (for headless scraping)
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


# Scrapper Code here

In [ ]:
def web_scrapper():
    # Initialize the WebDriver (Make sure you have the correct path for chromedriver)
    wd = webdriver.Chrome()

    # Base URL
    base_url = "https://basenotes.com/reviews/?&page={}"

    # Set the number of pages to scrape
    number_of_pages = 1000

    # Initialize an empty list to store reviews from all pages
    all_reviews = []

    # Loop through each page
    for page in range(1, number_of_pages + 1):
        # Navigate to the page
        wd.get(base_url.format(page))

        try:
            # Wait until the reviews section is visible
            WebDriverWait(wd, 2).until(
                EC.visibility_of_element_located((By.CLASS_NAME, 'fragreviewouter'))  # Class for the entire review block
            )

            # Find all review blocks on the page
            review_blocks = wd.find_elements(By.CLASS_NAME, 'fragreviewouter')

            # Loop through each review block
            for review_block in review_blocks:

                # Extract perfume name
                perfume_name = review_block.find_element(By.TAG_NAME, 'h3').text

                # Extract review text
                review_text = review_block.find_element(By.CLASS_NAME, 'fragreview').text

                # Extract review rating (Assuming the thumbs3.png indicates the rating, adapt if needed)
                rating_image = review_block.find_element(By.CLASS_NAME, 'reviewrating').find_element(By.TAG_NAME, 'img')
                rating_src = rating_image.get_attribute('src')
                if 'thumbs3.png' in rating_src:
                    rating = 1  # Adjust based on actual image mappings
                elif 'thumbs1.png' in rating_src:
                    rating = -1  # Adjust based on actual image mappings
                elif 'thumbs2.png' in rating_src:
                    rating = 0  # Adjust based on actual image mappings

                # Extract date
                review_date = review_block.find_element(By.CLASS_NAME, 'fragreviewfooter').text.split('\n')[0].strip()

                # Store data in a dictionary
                review_data = {
                    'page' : page,
                    'perfume': perfume_name,
                    'review': review_text,
                    'rating': rating,
                    'date': review_date

                }

                # Append the review to the list
                all_reviews.append(review_data)

        except Exception as e:
            print(f"An error occurred on page {page}: {e}")

    # Convert to DataFrame after all pages are scraped
    df = pd.DataFrame(all_reviews)

    # Display or save the data
    print(df)

    # If you want to save to CSV
    df.to_csv('scraped_reviews.csv', index=False)

    # Close the WebDriver
    wd.quit()


# Alex Code Below

In [ ]:
#scrapper_file = 'scraped_reviews.csv'
def basenotes_review_cleaner(file,min_review_flag=False, min_review_count=100 ):
    all_reviews = pd.read_csv(file)
    all_reviews['perfume_brand'] = all_reviews['perfume'].apply(lambda x: x.split('by ')[1] if 'by' in x else 'null')#.split('by ')[1]
    all_reviews['perfumes'] = all_reviews['perfume'].apply(lambda x: x.split('by ')[0] if 'by' in x else 'null')
    # all_reviews.drop(columns=['perfume','page'], inplace=True)
    # change the location of the all_reviews columns
    all_reviews = all_reviews[['perfume_brand', 'perfumes', 'review', 'rating', 'date']]
    if min_review_flag:
        # Filter brands with at least 50 reviews
        dfffff = all_reviews.groupby(['perfume_brand']).agg({'review': 'count'}).sort_values(by='review', ascending=False)
        dfffff = dfffff[dfffff['review'] >= min_review_count]
        all_reviews = all_reviews[all_reviews['perfume_brand'].isin(dfffff.index)]
    return all_reviews


#all_reviews_df.head()

In [ ]:
def preprocess(text):
    # Normalize unicode characters (e.g., accents)
    text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore')

    # Remove punctuation and convert to lowercase
    text_cleaned = re.sub(f"[{re.escape(string.punctuation)}]", "", text.lower())

    # Tokenize the text
    tokens = word_tokenize(text_cleaned)

    # Remove stopwords
    tokens = [word for word in tokens if word not in stopwords.words("english")]

    return tokens, " ".join(tokens)  # Return both tokens and cleaned string

In [ ]:
def topic_modeling_bertopic(reviews_df):
    # Preprocess the reviews
    processed_data = reviews_df['review'].apply(preprocess)

    # Split the result into two columns: 'tokens' and 'preprocessed_review'
    reviews_df['tokens'], reviews_df['preprocessed_review'] = zip(*processed_data)

    # Initialize the BERTopic model with the default transformer model
    topic_model = BERTopic(nr_topics = 'auto')  # No sentence transformer model specified

    # Ensure we are passing a list of strings
    preprocessed_reviews = reviews_df['preprocessed_review'].astype(str).tolist()

    # Fit the BERTopic model to the reviews
    topics, probabilities = topic_model.fit_transform(preprocessed_reviews)

    # Add the topics and probabilities to the DataFrame
    reviews_df['dominant_topic'] = topics
    reviews_df['topic_probabilities'] = probabilities.tolist()  # Ensure it's a list

    # Get topic information
    topic_info = topic_model.get_topic_info()

    return reviews_df, topic_info, topic_model

In [ ]:
#this combines the 2 above

def get_combined_topic_distribution_and_ratings(df, num_topics):
    # Ensure the dominant topics are included in the DataFrame
    if 'dominant_topic' not in df.columns:
        raise ValueError("The DataFrame must contain a 'dominant_topic' column.")

    # Group by 'perfume_brand' and 'perfumes' and filter perfumes with at least 10 reviews
    grouped = df.groupby(['perfume_brand', 'perfumes']).filter(lambda x: len(x) >= 10)

    # Initialize lists to hold the topic distribution and average ratings
    topic_distribution = []
    avg_ratings = []

    for (brand, perfume), group in grouped.groupby(['perfume_brand', 'perfumes']):
        # Get the count of reviews per dominant topic
        topic_counts = group['dominant_topic'].value_counts(normalize=True).to_dict()

        # Initialize row for the perfume
        row = {
            'brand': brand,
            'perfume': perfume,
            'num_reviews': len(group),  # Number of reviews
        }

        # Add the percentage of reviews for each topic
        for i in range(num_topics):
            row[f'topic_{i}_percentage'] = topic_counts.get(i, 0)  # Set to 0 if no reviews for this topic

        topic_distribution.append(row)

        # Calculate the average rating for the current perfume across all dominant topics
        avg_rating_row = {
            'brand': brand,
            'perfume': perfume,
            'avg_rating': group['rating'].mean(),  # Average rating
            'num_reviews': len(group)  # Number of reviews
        }

        avg_ratings.append(avg_rating_row)

    # Convert both lists to DataFrames
    topic_distribution_df = pd.DataFrame(topic_distribution)
    avg_ratings_df = pd.DataFrame(avg_ratings)

    # Merge the two DataFrames on 'brand' and 'perfume'
    combined_df = pd.merge(topic_distribution_df
                           , avg_ratings_df, on=['brand', 'perfume'], suffixes=('_topic', '_rating'))

    return combined_df



In [ ]:
def find_closest_topic(input_text, topic_model):
    """
    Preprocesses the input text, transforms it using a trained BERTopic model,
    and returns the closest topic along with the top words for that topic.

    Args:
    - input_text (str): The new text to classify into a topic.
    - topic_model (BERTopic): The trained BERTopic model.

    Returns:
    - most_probable_topic (int): The topic ID that the input text is closest to.
    - topic_words (list): The top words associated with the predicted topic.
    - topic_probabilities (list): The probabilities associated with all topics.
    """

    # Step 1: Preprocess the input text (assuming basic preprocessing here)
    # Preprocess the input
    preprocessed_input = preprocess(input_text)

    # Ensure it's a list as BERTopic expects a list of texts
    preprocessed_input_list = [preprocessed_input]

    # Step 2: Transform the input text using the trained BERTopic model
    topics, topic_probabilities = topic_model.transform(preprocessed_input_list)

    # Find the most probable topic (with the highest probability)
    most_probable_topic = topics[0]

    # Step 3: Get the top words for the predicted topic
    topic_words = topic_model.get_topic(most_probable_topic)

    return most_probable_topic, topic_words, topic_probabilities

# Alex code is above and Pranav code is below.

In [ ]:
import csv
def user_input_for_attribues():
    attributes = []
    while True:
        attribute = input("Enter attribute name (press 'q' to stop): ")
        if attribute == 'q':
            break
        else:
            attributes.append(attribute)

    # Save attributes to CSV file with each attribute on a new line
    with open('attributes.csv', 'w', newline='') as file:
        writer = csv.writer(file)
        for attribute in attributes:
            writer.writerow([attribute])
    return attributes


def preprocess_text(text):
    """
    Preprocess the input text: lowercasing, removing punctuation, and stopwords.

    Parameters:
        text (str): The input text to preprocess.

    Returns:
        str: The cleaned and preprocessed text.
    """
    stop_words = set(stopwords.words('english'))

    # Convert text to lowercase
    text = text.lower()

    # Remove punctuation
    text = text.translate(str.maketrans('', '', string.punctuation))

    # Remove stopwords
    text = " ".join([word for word in text.split() if word not in stop_words])

    return text

def calculate_similarity(df_original, product_column:str, review_column:str, attributes):
    """
    Calculate cosine similarity between product attributes and reviews using a bag-of-words model.

    Parameters:
        df (pd.DataFrame): The input DataFrame containing reviews.
        review_column (str): The name of the column containing text reviews.
        attributes (list): List of three predefined attributes for the product.
    Uses preprocess_text function to preprocess the reviews.
    Returns:
        pd.DataFrame: DataFrame with preprocessed review and similarity score.
    """
    df = df_original.copy()
    # Preprocess the reviews and add to a new column
    df['preprocessed_review'] = df[review_column].apply(preprocess_text)

    # Combine the attributes into a single string
    attributes_text = " ".join([preprocess_text(attr) for attr in attributes])

    # Initialize the TF-IDF vectorizer (Bag-of-Words model)
    vectorizer = TfidfVectorizer()

    # Fit and transform both the reviews and the attributes
    all_texts = [attributes_text] + df['preprocessed_review'].tolist()  # First element is the attributes, rest are reviews
    tfidf_matrix = vectorizer.fit_transform(all_texts)

    # Compute cosine similarity between attributes (first row) and all reviews (remaining rows)
    similarities = cosine_similarity(tfidf_matrix[0:1], tfidf_matrix[1:]).flatten()

    # Prepare the output DataFrame with similarity scores
    df['similarity_score'] = similarities

    return df[[product_column, 'preprocessed_review', 'similarity_score']]

def get_sentiment(df, df_col): #input df and col, return df with new col
    sid = SentimentIntensityAnalyzer()
    df_copy = df.copy()
    df_copy['sentiment'] = df_copy[df_col].apply(lambda x: sid.polarity_scores(x)['compound'])
    return df_copy

def calculate_similarity_spacy(df_original, product_column:str, review_column: str, attributes):
    """
    Calculate cosine similarity between product attributes and reviews using spaCy's word vectors.

    Parameters:
        df (pd.DataFrame): The input DataFrame containing reviews.
        review_column (str): The name of the column containing text reviews.
        attributes (list): List of three predefined attributes for the product.

    Returns:
        pd.DataFrame: DataFrame with preprocessed review and similarity score.
    """
    df = df_original.copy()
    # Preprocess the reviews and add to a new column
    df['preprocessed_review'] = df[review_column].apply(preprocess_text)

    # Combine the attributes into a single string
    attributes_text = " ".join([preprocess_text(attr) for attr in attributes])

    # Load the spaCy model with word vectors
    nlp = spacy.load("en_core_web_md")

    # Process the attributes text and get the vector
    attributes_vector = nlp(attributes_text).vector

    # Calculate similarity for each review
    similarities = []
    for review in df['preprocessed_review']:
        review_vector = nlp(review).vector  # Get the vector for the review
        similarity = cosine_similarity([attributes_vector], [review_vector]).flatten()[0]
        similarities.append(similarity)

    # Add the similarity score to the DataFrame
    df['similarity_score'] = similarities

    return df[[product_column, 'preprocessed_review', 'similarity_score']]


def calculate_percentage_mentions_v1(df,df_brand:str,df_review:str, product_name, attribute):
    """Calculates the percentage of reviews for a specific product that mention a given attribute."""
    product_reviews = df[df[df_brand] == product_name]
    reviews_with_attribute = product_reviews[product_reviews[df_review].str.contains(attribute, na=False)]
    percentage = (len(reviews_with_attribute) / len(product_reviews)) * 100 if len(product_reviews) > 0 else 0
    return percentage


In [ ]:
def get_bow_similarity(df, attributes_csvfile, recommendation_basis = 'perfumes', infunction_print_flag = True):

    attributes = pd.read_csv(attributes_csvfile, header=None).iloc[:,0].tolist()
    top_reviews_similarities_filename_output = "top_reviews_similarities_task_c.csv"

    reviews_similarities = calculate_similarity(df, recommendation_basis, 'review', attributes) # this beer is the csv file

    top_similarities = reviews_similarities.sort_values(by='similarity_score', ascending=False).head()

    # changing the column names of top_similarities (output file) as the question's instructions
    top_similarities.columns = [recommendation_basis, 'product_review', 'similarity_score']

    if infunction_print_flag:
      # Display the result
      print(top_similarities)

    top_similarities.to_csv(top_reviews_similarities_filename_output, index=True)
    #proof that similarities are not 0... similarities will increase using word vectors in part F
    return reviews_similarities

def vader_sentiment_analysis(df, attributes_csvfile, recommendation_basis = 'perfumes', infunction_print_flag = True):
# def vader_sentiment_analysis(reviews_similarities, infunction_print_flag = True):

    # we are doing VADER here
    reviews_similarities = get_bow_similarity(df,attributes_csvfile,recommendation_basis,False)
    similarities_sentiments_df = get_sentiment(reviews_similarities, 'preprocessed_review')
    if infunction_print_flag:
      print(similarities_sentiments_df.head())

    return similarities_sentiments_df

def prdouct_bow_composite_score(df, attributes_csvfile, recommendation_basis = 'perfumes', top_n = 5, infunction_print_flag = True):
# def prdouct_bow_composite_score(similarities_sentiments_df,product_column:str, infunction_print_flag = True):

    bow_product_composite_score_filename_output = "bow_product_composite_score_filename_output_task_e.csv"

    similarities_sentiments_df = vader_sentiment_analysis(df, attributes_csvfile,recommendation_basis, False)

    similarities_sentiments_df['composite_score'] = similarities_sentiments_df['similarity_score'] * similarities_sentiments_df['sentiment']

    top_composite_bow = similarities_sentiments_df.sort_values(by='composite_score', ascending=False)
    top_composite_bow_mean = top_composite_bow.groupby(recommendation_basis).agg({'composite_score': 'mean'}).reset_index().sort_values(by='composite_score', ascending=False)
    top_composite_bow_mean.columns = [recommendation_basis, 'composite_evaluation_score_bow']

    top_composite_bow_mean.to_csv(bow_product_composite_score_filename_output, index=True)

    if infunction_print_flag:
      print(f"The top {top_n} recommendations for the customers are: \n",top_composite_bow_mean.iloc[0:top_n,:]) #top composite scores with bag of words approach

    return top_composite_bow_mean

def product_spacy_composite_scores(df, attributes_csvfile, recommendation_basis = 'perfumes', top_n=3, print_similarity_score_flag=True):
    # TASK f
    attributes = pd.read_csv(attributes_csvfile, header=None).iloc[:,0].tolist()
    spacy_product_composite_score_filename_output = "spacy_product_composite_score_filename_output_task_f.csv"

    reviews_similarities = calculate_similarity_spacy(df, recommendation_basis, 'review', attributes)
    similarities_sentiments_df_spacy = get_sentiment(reviews_similarities, 'preprocessed_review')
    similarities_sentiments_df_spacy['composite_score'] = similarities_sentiments_df_spacy['similarity_score'] * similarities_sentiments_df_spacy['sentiment']


    similarities_sentiments_df_spacy_mean = similarities_sentiments_df_spacy.groupby(recommendation_basis).agg({'composite_score': 'mean'}).reset_index().sort_values(by='composite_score', ascending=False)

    similarities_sentiments_df_spacy_mean.columns = [recommendation_basis,'composite_score_spacy']
    if print_similarity_score_flag:
        similarities_sentiments_df_spacy_mean.to_csv(spacy_product_composite_score_filename_output, index=True)

        print("\n--------------Calculating Percent of comments containing the attributes---------------")

        # ------------------------------------------------------------------
        #top 3 recommendations in 'top_3_highest_rated'
        for _, row in similarities_sentiments_df_spacy_mean.iloc[0:top_n,:].iterrows():
          product__name = row[recommendation_basis]
          for attribute in attributes:  # Loop through each attribute specified by the customer
              percentage = calculate_percentage_mentions_v1(df,recommendation_basis, 'review', product__name, attribute)
              print(f"For product '{product__name}', {percentage:.2f}% of reviews mention the attribute '{attribute}'.")
          print("\n")
        # ------------------------------------------------------------------

        print(f"The top {top_n} recommendations for the customers as per spaCy approach are: \n {similarities_sentiments_df_spacy_mean.iloc[0:top_n,:]}")


    return similarities_sentiments_df_spacy_mean

# Function to query SerpAPI for perfume prices and links
def get_perfume_prices_and_links(perfume_names_list, api_key = 'e6a66dccbdf2d75261e6a62055663ec25d4f8f339893638f945740f33429a652'):
    # Define the parameters for the API request
    for perfume_name in perfume_names_list:
        params = {
            "engine": "google_shopping",
            "q": perfume_name,
            "hl": "en",  # Language
            "gl": "us",  # Country
            "api_key": api_key
        }
        # Send a request to the SerpAPI
        response = requests.get("https://serpapi.com/search.json", params=params)
        # Check if the response is successful
        if response.status_code == 200:
            data = response.json()
            # Extract shopping results
            shopping_results = data.get("shopping_results", [])
            # Get the top 5 product links and prices
            top_5_results = shopping_results[:5]
            for i, result in enumerate(top_5_results):
                title = result.get("title", "N/A")
                price = result.get("extracted_price", "N/A")
                link = result.get("link", "N/A")
                store = result.get("source", "N/A")
                print(f"{i+1}. Title: {title}")
                print(f"   Price: ${price}")
                print(f"   Store: {store}")
                # print(f"   Link: {link}\n")
        else:
            print(f"Error: {response.status_code}, {response.text}")


## Running the code here all at once

In [ ]:
scrapper_file = 'scraped_reviews.csv'
all_reviews_df = basenotes_review_cleaner(scrapper_file,True,50)

In [ ]:
#cheap men products (cheap,men,woody,longlasting,attractive)

user_input_for_attribues()
r_basis = 'perfumes' # can also be 'perfume_brand'
reviews_similarities = get_bow_similarity(all_reviews_df, 'attributes.csv',recommendation_basis=r_basis,infunction_print_flag=False)
similarities_sentiments_df = vader_sentiment_analysis(all_reviews_df, 'attributes.csv',recommendation_basis=r_basis,infunction_print_flag=False)

prdouct_bow_composite_scores = prdouct_bow_composite_score(all_reviews_df,'attributes.csv',recommendation_basis=r_basis,top_n=5,infunction_print_flag=False)

f = product_spacy_composite_scores(all_reviews_df,"attributes.csv",recommendation_basis=r_basis, top_n=5)
# top_recommended_products = f[r_basis][:1].to_list() # can use "prdouct_bow_composite_scores" instead of "f"
# get_perfume_prices_and_links(top_recommended_products)
f


Enter attribute name (press 'q' to stop): woody
Enter attribute name (press 'q' to stop): fruity
Enter attribute name (press 'q' to stop): manly
Enter attribute name (press 'q' to stop): q


In [ ]:
# --- topic modeling functions ---
# all_reviews_df = basenotes_review_cleaner(scrapper_file,True,50)
cleaned_reviews_df, topic_info, topic_model = topic_modeling_bertopic(all_reviews_df) #calls preprocess also

num_topics = len(topic_info)  # Use the length of the topic_info to determine the number of topics
combined_df = get_combined_topic_distribution_and_ratings(cleaned_reviews_df, num_topics) #dopic distributions and avg ratings


# new_input = "woody fruity manly"

# # Assuming you have a trained topic_model from BERTopic
# predicted_topic, top_words, topic_probs = find_closest_topic(new_input, topic_model)

# print(f"Predicted Topic: {predicted_topic}")
# print(f"Top Words for Topic {predicted_topic}: {top_words}")
# print(f"Topic Probabilities: {topic_probs}")


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Predicted Topic: -1
Top Words for Topic -1: [('like', 0.007311707473283924), ('scent', 0.005973594215228826), ('one', 0.00573393741638168), ('fragrance', 0.005665275404963921), ('de', 0.005358683657213874), ('notes', 0.005201604048794651), ('smell', 0.005151527249713516), ('perfume', 0.004951858589052679), ('bit', 0.004941846185609374), ('get', 0.0049111198553461735)]
Topic Probabilities: [0.]


In [ ]:
#combined df from topic modeling
combined_df.head()

,brand,perfume,num_reviews_topic,topic_0_percentage,topic_1_percentage,topic_2_percentage,topic_3_percentage,topic_4_percentage,topic_5_percentage,topic_6_percentage,...,topic_91_percentage,topic_92_percentage,topic_93_percentage,topic_94_percentage,topic_95_percentage,topic_96_percentage,topic_97_percentage,topic_98_percentage,avg_rating,num_reviews_rating
0,Akro,Awake,13,0.769231,0.0,0.0,0.0,0.000000,0.0,0.0,...,0,0.000000,0.0,0,0.000000,0.0,0,0,0.769231,13
1,Amouage,Beach Hut Man,11,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,...,0,0.000000,0.0,0,0.000000,0.0,0,0,0.454545,11
2,Amouage,Bracken Man,11,0.272727,0.0,0.0,0.0,0.000000,0.0,0.0,...,0,0.090909,0.0,0,0.454545,0.0,0,0,0.545455,11
3,Amouage,Enclave,10,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,...,0,0.000000,0.0,0,0.000000,0.0,0,0,-0.300000,10
4,Amouage,Interlude Man,14,0.428571,0.0,0.0,0.0,0.071429,0.0,0.0,...,0,0.000000,0.0,0,0.000000,0.0,0,0,0.428571,14


In [ ]:

new_input = "woody fruity manly"

# Assuming you have a trained topic_model from BERTopic
predicted_topic, top_words, topic_probs = find_closest_topic(new_input, topic_model)

print(f"Predicted Topic: {predicted_topic}")
print(f"Top Words for Topic {predicted_topic}: {top_words}")
print(f"Topic Probabilities: {topic_probs}")

Predicted Topic: -1
Top Words for Topic -1: [('like', 0.007311707473283924), ('scent', 0.005973594215228826), ('one', 0.00573393741638168), ('fragrance', 0.005665275404963921), ('de', 0.005358683657213874), ('notes', 0.005201604048794651), ('smell', 0.005151527249713516), ('perfume', 0.004951858589052679), ('bit', 0.004941846185609374), ('get', 0.0049111198553461735)]
Topic Probabilities: [0.]


In [ ]:
new_input = "mint"

# Assuming you have a trained topic_model from BERTopic
predicted_topic, top_words, topic_probs = find_closest_topic(new_input, topic_model)

print(f"Predicted Topic: {predicted_topic}")
print(f"Top Words for Topic {predicted_topic}: {top_words}")
print(f"Topic Probabilities: {topic_probs}")

Predicted Topic: -1
Top Words for Topic -1: [('like', 0.007311707473283924), ('scent', 0.005973594215228826), ('one', 0.00573393741638168), ('fragrance', 0.005665275404963921), ('de', 0.005358683657213874), ('notes', 0.005201604048794651), ('smell', 0.005151527249713516), ('perfume', 0.004951858589052679), ('bit', 0.004941846185609374), ('get', 0.0049111198553461735)]
Topic Probabilities: [0.]


In [ ]:
# all the graphs here
fig = topic_model.visualize_hierarchy()
fig.write_html("topic_hierarchy.html")  # Save the figure as an HTML file
fig.show()

fig = topic_model.visualize_topics()
fig.write_html("topic_visualization.html")  # Save the figure as an HTML file
fig.show()

fig = topic_model.visualize_heatmap()
fig.write_html("topic_heatmap.html")  # Save the figure as an HTML file
fig.show()

# Visualize topics with barchart
fig = topic_model.visualize_barchart()
fig.write_html("topic_barchart.html")  # Save the figure as an HTML file
fig.show()


In [ ]:
print((num_topics))

99
